학습용 영어-프랑스어 병렬 문장 데이터 준비
개념: 
   - 입력(영어)과 출력(프랑스어) 쌍으로 구성
   - 디코더 입력에는 시작 토큰(\t), 타겟에는 종료 토큰(\n) 추가
 설명:
   - input_texts: 인코더에 입력될 영어 문장
   - target_texts: 디코더가 생성해야 할 프랑스어 문장 (전처리 포함)

In [19]:
import numpy as np
import tensorflow as tf

data_pairs = [
    ("Hello", "Bonjour"),
    ("How are you", "Comment allez-vous"),
    ("Good morning", "Bonjour matin"),
    ("Thank you", "Merci"),
]

# 입력과 타켓을 분리
input_texts = []    
target_texts = []

# 입력에는 시작토큰, 타겟에는 종료토큰을 부여
for eng, fra in data_pairs:
    input_texts.append(eng)
    # 디코더 입력 '\t'(시작), 디코더 출력 '\n'(종료)
    target_texts.append(f'\t{fra}\n')
for i in range(len(input_texts)):
    print(f"입력:{input_texts[i]:20s} --> 타겟 : {target_texts[i]}")

입력:Hello                --> 타겟 : 	Bonjour

입력:How are you          --> 타겟 : 	Comment allez-vous

입력:Good morning         --> 타겟 : 	Bonjour matin

입력:Thank you            --> 타겟 : 	Merci



문자 단위 사전(vocabulary) 생성 및 정수 인덱스 변환
- 개념:
    - 각 문자를 고유한 정수로 매핑
    - 입력과 타겟의 사전은 별도 관리
    - 원-핫 인코딩으로 신경망 입력 형태 생성
- 설명:
    - input_characters: 영어 문장에 등장하는 모든 고유 문자
    - target_characters: 프랑스어 문장 + 특수 토큰(\t, \n)
    - encoder_input_data: 3D 배열 (샘플, 시퀀스 길이, 문자 사전 크기)

In [25]:
# 입력과 타겟의 고유한 문자 수집
# input_characters = set()
# target_characters = set()

# for text in input_texts:
#     for char in text:
#         input_characters.add(char)

input_characters = {char for text in input_texts for char in text}
target_characters = {char for target_text in target_texts  for char in target_text}

# 정렬해서 일관성 확보
input_characters = sorted(list(input_characters))
target_characters= sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# 가장 긴 문장 길이 계산
max_encoder_seq_length = max(len(txt) for txt in input_texts  )
max_decoder_seq_length = max(len(txt) for txt in target_texts  )
# 문자-> 인덱스 매핑
input_token_index = { char:i for i,char in enumerate(input_characters)}
target_token_index = { char:i for i,char in enumerate(target_characters)}

# 인덱스 -> 문자 역매핑(추론시 사용)
reverse_input_token_index =  { idx:char for char,idx in input_token_index.items()}
reverse_target_token_index = { idx:char for char,idx in target_token_index.items()}
#  encoder_input_data: 3D 배열 (샘플, 시퀀스 길이, 문자 사전 크기)
encoder_input_data = np.zeros( (len(input_texts),max_encoder_seq_length,num_encoder_tokens ),
                              dtype='float32' )
decoder_input_data = np.zeros( (len(input_texts),max_decoder_seq_length,num_decoder_tokens ),
                              dtype='float32' )
decoder_target_data = np.zeros( (len(input_texts),max_decoder_seq_length,num_decoder_tokens ),
                              dtype='float32' )

# 문자별 원핫 인코딩
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0    
    for t, char in enumerate(target_text):
        # decoder_input_data: 전체 타겟 시퀀스 (시작 토큰 포함)
        decoder_input_data[i, t, target_token_index[char]] = 1.0        
        # decoder_target_data: 한 타임스텝 앞선 정답 (Teacher Forcing용)
        # 디코더 입력 \t안녕
        # 디코더 출력 안녕\n
        # 한 스텝 시프트 -  Teacher Forcing
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
# hi  입력
# \t hello  디코더 입력
# hello \n  디코더 출력 - Teacher Forcing  한스텝 앞으로 이동
print(f'고유 입력 문자수 : {num_encoder_tokens}')
print(f'고유 타겟 문자수 : {num_decoder_tokens}')
print(f'최대 입력 문장길이 : {max_encoder_seq_length}')
print(f'최대 타겟 문장길이 : {max_decoder_seq_length}')
print('# 샘플, 시퀀스 길이, 문자 사전 크기')
print(f'encoder_input_data : {encoder_input_data.shape}')
print(f'decoder_input_data : {decoder_input_data.shape}')
print(f'decoder_target_data : {decoder_target_data.shape}')

고유 입력 문자수 : 19
고유 타겟 문자수 : 22
최대 입력 문장길이 : 12
최대 타겟 문장길이 : 20
# 샘플, 시퀀스 길이, 문자 사전 크기
encoder_input_data : (4, 12, 19)
decoder_input_data : (4, 20, 22)
decoder_target_data : (4, 20, 22)


LSTM 기반 Seq2Seq 인코더-디코더 학습 모델 구축
- 개념:
    - Encoder: 입력 시퀀스를 처리하고 최종 상태(h, c) 출력
    - Decoder: Encoder 상태를 초기값으로 받아 타겟 시퀀스 생성
    - return_state=True: LSTM 내부 상태(h, c) 반환
    - return_sequences=True: 모든 타임스텝 출력
- 설명:
    - encoder_states: [h, c] (hidden state, cell state)
    - decoder_lstm: 초기 상태로 encoder_states 전달
    - decoder_dense: Softmax로 각 타임스텝의 문자 확률 분포 생성

In [ ]:
latent_dim = 256  # LSTM 은닉 차원 (내부 표현 크기)

# ==================== Encoder ====================
encoder_inputs = input(shape=(None, num_encoder_tokens), name='encoder_input')
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# encoder_outputs는 사용하지 않고, 내부 상태(state_h, state_c)만 디코더로 전달
encoder_states = [state_h, state_c]

# ==================== Decoder ====================
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_input')
decoder_lstm = LSTM(latent_dim, return_sequences=True,s return_state=True, name='decoder_lstm')

# 디코더 초기 상태로 인코더 최종 상태 사용 (컨텍스트 전달)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# 각 타임스텝에서 문자 확률 분포 생성
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# ==================== 학습 모델 ====================
model = Model([encoder_inputs, decoder_inputs], decoder_outputs, name='seq2seq_training')

print("\n 모델 구조:")
model.summary()